In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.datasets import fashion_mnist
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau

2024-05-15 16:49:43.117069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(train_features, train_labels), (test_features, test_labels) = fashion_mnist.load_data()

In [3]:
train_features = np.expand_dims(train_features, axis=-1)
test_features = np.expand_dims(test_features, axis=-1)

In [4]:
train_features, validation_features, train_labels, validation_labels = \
    train_test_split(train_features, train_labels, test_size=0.2, random_state=0)

In [5]:
X_train, y_train = train_features, to_categorical(train_labels)
X_val, y_val = validation_features, to_categorical(validation_labels)

In [6]:
def make_cnn():
    
    model = Sequential()

    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(6, (5, 5), activation='tanh', padding='same'))
    model.add(Dropout(0.05))
    model.add(AveragePooling2D((2, 2), strides=2))
    model.add(Conv2D(32, (5, 5), activation='tanh', padding='valid'))
    model.add(Dropout(0.05))
    model.add(AveragePooling2D((2, 2), strides=2))
    model.add(Conv2D(128, (5, 5), activation='tanh', padding='valid'))
    model.add(Dropout(0.05))
    model.add(Flatten())
    model.add(Dense(120, activation='linear'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(BatchNormalization())
    model.add(Dropout(0.05))
    model.add(Dense(84, activation='linear'))
    model.add(LeakyReLU(alpha=0.02))
    model.add(BatchNormalization())
    model.add(Dropout(0.05))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

    return model

In [7]:
model = make_cnn()

2024-05-15 16:49:44.970162: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 dropout (Dropout)           (None, 28, 28, 6)         0         
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 32)        4832      
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 32)        0         
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 32)         0         
 ePooling2D)                                            

In [11]:
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=0,
    restore_best_weights=True
)

reduceLr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.2,
    patience=5,
    mode="min",
    verbose=0,
    min_delta=0.0001,
    min_lr=0
)

tb = TensorBoard(log_dir="./logs")

cb = [earlystop, reduceLr, tb]

In [15]:
model.fit(
    X_train,
    y_train,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=cb,
    batch_size=64
)

Epoch 1/20
157/157 [==============================] - 16s 102ms/step - loss: 0.2228 - accuracy: 0.9161 - val_loss: 0.3489 - val_accuracy: 0.8822 - lr: 0.0010
Epoch 2/20
 36/157 [=====>........................] - ETA: 9s - loss: 0.2237 - accuracy: 0.9188

In [ ]:
y_pred = model.predict(test_features)

In [ ]:
accuracy_score(test_labels, np.argmax(y_pred, axis=1))

In [ ]:
def display_image(image, title):
    image = image.squeeze()
    plt.figure()
    plt.title(title)
    plt.imshow(image, cmap=plt.cm.gray_r)

In [ ]:
display_image(test_features[0], test_labels[0])